In [ ]:
# yolov5x6 trained for 300 epochs.

In [1]:
!nvidia-smi

Sun Sep 18 21:42:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%capture
!git clone https://github.com/ultralytics/yolov5 
%cd yolov5
!pip install -r requirements.txt wandb

In [3]:
from PIL import Image, ExifTags
from pycocotools.coco import COCO
from matplotlib.patches import Polygon, Rectangle
from matplotlib.collections import PatchCollection
import colorsys
import random
import pylab

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from tqdm import tqdm

import shutil
import os

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [4]:
%%capture
%cd ..
!gdown 1X3O2v3GIPveq3ylWF6o1qHI5uzbN1vWA
!unzip rotated2.zip
%mv /content/content/* /content

In [5]:
#@title
# dataset_path = './data'
# anns_file_path = dataset_path + '/' + 'annotations.json'

# # Read annotations
# with open(anns_file_path, 'r') as f:
#     dataset = json.loads(f.read())

# categories = dataset['categories']
# anns = dataset['annotations']
# imgs = dataset['images']
# nr_cats = len(categories)
# nr_annotations = len(anns)
# nr_images = len(imgs)

# # Load categories and super categories
# cat_names = []
# super_cat_names = []
# super_cat_ids = {}
# super_cat_last_name = ''
# nr_super_cats = 0
# for cat_it in categories:
#     cat_names.append(cat_it['name'])
#     super_cat_name = cat_it['supercategory']
#     # Adding new supercat
#     if super_cat_name != super_cat_last_name:
#         super_cat_names.append(super_cat_name)
#         super_cat_ids[super_cat_name] = nr_super_cats
#         super_cat_last_name = super_cat_name
#         nr_super_cats += 1

# print('Number of super categories:', nr_super_cats)
# print('Number of categories:', nr_cats)
# print('Number of annotations:', nr_annotations)
# print('Number of images:', nr_images)

In [6]:
# # Read annotations
# dataset_path = './data'
# anns_file_path = dataset_path + '/' + 'annotations.json'

# with open(anns_file_path, 'r') as f:
#     dataset = json.loads(f.read())
# imgs = dataset['images']

# data_source = COCO(annotation_file=anns_file_path)

In [7]:
!pwd

/content


In [8]:
## train test split
'''
train: images/train
val: images/val
test: images/test
'''
import random
id_list=[i for i in range(1500)]
random.shuffle(id_list)

train_ids = id_list[:1050]
val_ids = id_list[1050:1050+300]
test_ids = id_list[1050+300:]

def move_helper(ids, desti):
  for id in ids:
    img_name = os.path.join( './yoloTACO/images', str(id)+'.jpg' )
    lbl_name = os.path.join( './yoloTACO/labels', str(id)+'.txt' )
    print(img_name)
    if os.path.isfile(img_name):
        shutil.move( img_name, 'yoloTACO/images/'+desti)
        shutil.move( lbl_name, 'yoloTACO/labels/'+desti)
    else :
        print('file does not exist', img_name)

In [9]:
%%capture
!mkdir yoloTACO/images/train
!mkdir yoloTACO/images/val
!mkdir yoloTACO/images/test
!mkdir yoloTACO/labels/train
!mkdir yoloTACO/labels/val
!mkdir yoloTACO/labels/test
move_helper(test_ids,'test')
move_helper(train_ids,'train')
move_helper(val_ids,'val')

In [10]:
#@title yml

with open('/content/yolov5/data/yoloTACO.yaml', mode='w') as fp:
  lines = '''path: ../yoloTACO  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
test: images/test # test images (optional)

# Classes
names:
  0: Aluminium foil
  1: Battery
  2: Aluminium blister pack
  3: Carded blister pack
  4: Other plastic bottle
  5: Clear plastic bottle
  6: Glass bottle
  7: Plastic bottle cap
  8: Metal bottle cap
  9: Broken glass
  10: Food Can
  11: Aerosol
  12: Drink can
  13: Toilet tube
  14: Other carton
  15: Egg carton
  16: Drink carton
  17: Corrugated carton
  18: Meal carton
  19: Pizza box
  20: Paper cup
  21: Disposable plastic cup
  22: Foam cup
  23: Glass cup
  24: Other plastic cup
  25: Food waste
  26: Glass jar
  27: Plastic lid
  28: Metal lid
  29: Other plastic
  30: Magazine paper
  31: Tissues
  32: Wrapping paper
  33: Normal paper
  34: Paper bag
  35: Plastified paper bag
  36: Plastic film
  37: Six pack rings
  38: Garbage bag
  39: Other plastic wrapper
  40: Single-use carrier bag
  41: Polypropylene bag
  42: Crisp packet
  43: Spread tub
  44: Tupperware
  45: Disposable food container
  46: Foam food container
  47: Other plastic container
  48: Plastic glooves
  49: Plastic utensils
  50: Pop tab
  51: Rope & strings
  52: Scrap metal
  53: Shoe
  54: Squeezable tube
  55: Plastic straw
  56: Paper straw
  57: Styrofoam piece
  58: Unlabeled litter
  59: Cigarette'''
  fp.writelines(lines)

In [11]:
%cd ./yolov5
!ls

/content/yolov5
benchmarks.py	 detect.py   models	       setup.cfg       val.py
classify	 export.py   README.md	       train.py
CONTRIBUTING.md  hubconf.py  requirements.txt  tutorial.ipynb
data		 LICENSE     segment	       utils


In [12]:
# check if we have
!pip list -v | grep [Aa]lbumentations

albumentations                1.2.1                        /usr/local/lib/python3.7/dist-packages pip


In [ ]:
# Train
!python train.py --img 1280 \
 --batch-size 4 --epochs 300 \
 --data yoloTACO.yaml \
 --weights yolov5x6.pt \
# --multi-scale \
# --freeze 10


wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5x6.pt, cfg=, data=yoloTACO.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=4, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-137-gfda8aa5 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=

In [ ]:
from utils.plots import plot_results 
plot_results('/content/yolov5/runs/train/exp/results.csv') 

In [ ]:
from IPython import display
display.Image("./runs/train/exp/results.png")

In [ ]:
!python val.py --data yoloTACO.yaml --weights runs/train/exp/weights/best.pt